<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, Input, MaxPool2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
(ds_train, ds_test), ds_info = tfds.load('cifar100', split = ['train', 'test'], shuffle_files=True,
                             as_supervised = True, with_info = True,batch_size=16)

ds_train_subset = ds_train.take(1000)
# ds_train_subset = ds_train_subset.batch(16)
ds_val = ds_train.skip(1100)
# ds_val = ds_val.batch(16)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incomplete4JV0S0/cifar100-train.tfrecord*...:   0%|        …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incomplete4JV0S0/cifar100-test.tfrecord*...:   0%|         …

Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [44]:
# Create a subset of ds_train and ds_val
ds_train_subset = ds_train.take(1000)
ds_val = ds_train.skip(1100).take(100)

# Convert datasets to NumPy arrays
def dataset_to_numpy(dataset):
    features = []
    labels = []

    for feature, label in tfds.as_numpy(dataset):
        features.append(feature)
        labels.append(label)

    return np.array(features), np.array(labels)

# Convert ds_train_subset and ds_val to NumPy arrays
np_train_features, np_train_labels = dataset_to_numpy(ds_train_subset)
np_val_features, np_val_labels = dataset_to_numpy(ds_val)

In [45]:
print("-------TRAIN--------")
print(np_train_features.shape)
print(np_train_labels.shape)
print("-------VALIDATION--------")
print(np_val_features.shape)
print(np_val_labels.shape)

-------TRAIN--------
(1000, 16, 32, 32, 3)
(1000, 16)
-------VALIDATION--------
(100, 16, 32, 32, 3)
(100, 16)


In [46]:
np_test_features, np_test_labels = dataset_to_numpy(ds_test)
print("-------TEST--------")
print(np_test_features.shape)
print(np_test_labels.shape)

-------TEST--------
(625, 16, 32, 32, 3)
(625, 16)


In [40]:
CLASS_NUM = np_test_labels.shape[1]
print(CLASS_NUM)

16


In [57]:
tf.keras.backend.clear_session()
model = Sequential([
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Flatten(),

    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),

    Dense(CLASS_NUM,
          activation = 'softmax')

    ])
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'], run_eagerly=True)

In [58]:
model.fit(x = np_train_features.astype('float32'), y = np_train_labels, epochs = 3)

Epoch 1/3


ValueError: Exception encountered when calling layer 'sequential' (type Sequential).

Input 0 of layer "max_pooling2d" is incompatible with the layer: expected ndim=4, found ndim=5. Full shape received: (32, 16, 30, 30, 32)

Call arguments received by layer 'sequential' (type Sequential):
  • inputs=tf.Tensor(shape=(32, 16, 32, 32, 3), dtype=float32)
  • training=True
  • mask=None